# Calculating Commute Mode by Census Tract for Los Angeles County

In [1]:
# set up work environment
%matplotlib inline
import pandas as pd, numpy as np, matplotlib.pyplot as plt
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point
from scipy import ndimage
import matplotlib.pylab as pylab
import json
import requests
import pprint
import numpy as np
from mapboxgl.viz import *
from mapboxgl.utils import *
import ipywidgets as widgets

pylab.rcParams['figure.figsize'] = 10, 8

import warnings 
warnings.filterwarnings('ignore')

In [2]:
# bring in los angeles county census tracts
la_tracts = gpd.read_file('data/LA_tracts.geojson')
la_tracts.head()

,shape_area,label,x_center,ct10,y_center,geoid10,shape_len,geometry
0,4025735684.42,9110.01,6620403.0,911001,1998891.0,06037911001,353933.808192,(POLYGON ((-117.6671211134298 34.5580081383781...
1,2078689856.02,9800.03,6575300.0,980003,2112006.0,06037980003,273188.86321,(POLYGON ((-117.8806120004425 34.7636159996687...
2,11118018325.1,9303.01,6603027.0,930301,1932124.0,06037930301,628603.531323,(POLYGON ((-117.6552358388393 34.3972219664572...
3,4824001.88224,5730.03,6500215.0,573003,1747305.0,06037573003,9050.00845755,(POLYGON ((-118.1992330000361 33.7971229999225...
4,6697030.7108,2976.02,6473372.0,297602,1719119.0,06037297602,12308.3153848,(POLYGON ((-118.2879799997457 33.7225829999421...


In [3]:
# convert numeric columns from string to integer
la_tracts[['ct10']] = la_tracts[['ct10']].astype(int)

In [4]:
commute = pd.read_csv('data/losangeles_commute.csv')

In [6]:
# combine the la tract geography with the commute mode data
la_commute = la_tracts.merge(commute, left_on='ct10', right_on='tract')
# drop the rows with null values to ensure the data will get mapped
la_commute = la_commute.dropna(subset=['Percent_Drove','Percent_Transit','Percent_Bike','Percent_Walk','Percent_WFH'])
# view the output
la_commute.head()

,shape_area,label,x_center,ct10,y_center,geoid10,shape_len,geometry,Geography,fips_code,...,Drove,Percent_Drove,Transit,Percent_Transit,Bike,Percent_Bike,Walk,Percent_Walk,WFH,Percent_WFH
0,4025735684.42,9110.01,6620403.0,911001,1998891.0,06037911001,353933.808192,(POLYGON ((-117.6671211134298 34.5580081383781...,"Census Tract 9110.01, Los Angeles County, Cali...",6037911001,...,1246,85.52,0.0,0.00,0.0,0.00,0.0,0.00,211.0,14.48
1,2078689856.02,9800.03,6575300.0,980003,2112006.0,06037980003,273188.86321,(POLYGON ((-117.8806120004425 34.7636159996687...,"Census Tract 9800.03, Los Angeles County, Cali...",6037980003,...,0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00
2,11118018325.1,9303.01,6603027.0,930301,1932124.0,06037930301,628603.531323,(POLYGON ((-117.6552358388393 34.3972219664572...,"Census Tract 9303.01, Los Angeles County, Cali...",6037930301,...,213,85.89,7.0,2.82,5.0,2.02,3.0,1.21,15.0,6.05
3,4824001.88224,5730.03,6500215.0,573003,1747305.0,06037573003,9050.00845755,(POLYGON ((-118.1992330000361 33.7971229999225...,"Census Tract 5730.03, Los Angeles County, Cali...",6037573003,...,732,82.81,65.0,7.35,0.0,0.00,0.0,0.00,51.0,5.77
4,6697030.7108,2976.02,6473372.0,297602,1719119.0,06037297602,12308.3153848,(POLYGON ((-118.2879799997457 33.7225829999421...,"Census Tract 2976.02, Los Angeles County, Cali...",6037297602,...,1672,90.23,37.0,2.00,13.0,0.70,0.0,0.00,66.0,3.56


In [7]:
# convert numeric columns from string to integer
la_commute[['Percent_Drove','Percent_Transit','Percent_Bike','Percent_Walk','Percent_WFH']] = la_commute[['Percent_Drove','Percent_Transit','Percent_Bike','Percent_Walk','Percent_WFH']].astype(int)

In [14]:
# export the commute data to be used on Carto
la_commute.to_csv('data/losangeles_commute.csv')
la_commute.to_file('data/la_commute.geojson', driver='GeoJSON')